In [75]:
from math import sqrt, floor
import collections.abc as abc
import secrets
from copy import deepcopy as cp

import numpy as np
from numpy.lib.recfunctions import unstructured_to_structured as unstruc_to_struc

import graphviz as gv
import imageio.v2 as imageio

In [76]:
# to reproduce
seed = 142917470789520928980186640852070266041
# seed = secrets.randbits(128)
rng = np.random.default_rng(seed)


class Graph:
  def __init__(self, n_nodes: int = 20):
    # generate random node position
    self.n_nodes = n_nodes
    self.nodes = rng.integers(low=0., high=100., size=(self.n_nodes, 2))
    self.nodes = unstruc_to_struc(self.nodes, dtype=np.dtype([('x', int), ('y', int)]))
    self.nodes = np.sort(self.nodes, order=['x', 'y'])
    
    self.dist = np.zeros(shape=(self.n_nodes, self.n_nodes))
    for r in range(self.n_nodes):
      for c in range(r):
        self.dist[r][c] = self.dist[c][r]
      for c in range(r + 1, self.n_nodes):
        self.dist[r][c] = sqrt(
          (self.nodes[r]['x'] - self.nodes[c]['x']) ** 2 +
          (self.nodes[r]['y'] - self.nodes[c]['y']) ** 2
        )
  
  
  def distance(self, node1: int, node2: int) -> float:
    return self.dist[node1][node2]
    
    
  def route_len(self, route: abc.Sequence[int]) -> float:
    rlen = 0
    for i in range(1, self.n_nodes):
      rlen += self.dist[route[i]][route[i - 1]]
    rlen += self.dist[route[-1]][route[0]]
    return rlen
  
  
  # saves graph with the given route to ./route/route_{idx}.png
  def visualize(self, route: abc.Sequence[int], idx: int = 0):
    vis = gv.Graph('salesman_route', engine='neato', format='png')
    vis.attr(label=f'route_{idx}')
    
    for i in range(self.n_nodes):
      pos = f"{self.nodes[i]['x'] / 15},{self.nodes[i]['y'] / 15}!"
      vis.node(str(i), pos=pos, width='0', height='0')

    for i in range(1, self.n_nodes):
      vis.edge(str(route[i]), str(route[i - 1]), color='#ac4242')
    vis.edge(str(route[self.n_nodes - 1]), str(route[0]), color='#ac4242')

    vis.render(cleanup=True, directory='route', filename=f'route_{idx}')


def route_evolution_gif(evolution_depth: int, gifname_suffix: str = ''):
  images = []
  for idx in range(evolution_depth):  
    images.append(imageio.imread(f'route/route_{idx}.png'))
  imageio.mimsave(f'route_evolution{gifname_suffix}.gif', images)


graph = Graph()
init_route = list(range(graph.n_nodes))
rng.shuffle(init_route)

### Ants Colony Optimization

The algorithm simulates the behavior of ants. In nature, ants leave pheromone trails. The ant wanders randomly, unless it comes across a pheromone trail.In that case, it is more likely that it will follow the trail. The more pheromones there are, the more ants have passed through here, meaning this route is good enough.

The algorithm creates several generations (their number is controlled by the `n_gener` parameter) of ant colonies, one after the other. In each generation, except for the first, the ants rely on pheromones left behind by previous generations. From every node, several ants leave (`n_ants` parameter) in every generation. The ants in each node select the next node randomly with respect to the pheromone signals on the edges. Additionally, there are elite ants (`n_elites` parameter) who only follow the best routes (i.e., those with the most pheromone signals). First generation starts with equal pheromones on each edge (`pher_init`). From generation to generation, pheromones evaporate with a certain coefficient (`vapor_cf`). 

**Probability of ant moving from i-th to j-th node:**
$ P_{ij}=\cfrac{\tau_{ij}^\alpha\eta_{ij}^\beta}{\sum{\tau_{im}^\alpha\eta_{im}^\beta}} $

$\tau - \text{pheromones}$, 
$\eta - \text{inversed distance}$, 
$\alpha,\beta - \text{constant parameters}$


In [77]:
def ants_colony(graph: Graph, n_ants=2, n_elites=0.3, alfa=1, beta=1, pher_init=0.5,
                close_cf=1, vapor_cf=0.3, pher_delta_cf=1, pher_max=1, n_gener=1000) \
    -> abc.Sequence[int]:

  # pheromones for the first generation, all equals
  pheromone = np.full((graph.n_nodes, graph.n_nodes), pher_init)
  # the matrix of values inversed to the distance with certain coefficient
  closeness = np.zeros(shape=(graph.n_nodes, graph.n_nodes))
  for r in range(graph.n_nodes):
    for c in range(r):
      closeness[r][c] = closeness[c][r]
    for c in range(r + 1, graph.n_nodes):
      closeness[r][c] = close_cf / graph.distance(r, c)
  
  # each ant's route and the best (i.e. the shortest) route found so far
  # route end connected to the beginning so it's cycle
  curr_route, best_route = cp(init_route), cp(init_route)
  # best route length
  best_len = float('inf')
  
  # just a list of node indices to pass it to numpy.random.choice
  next_pos_choices = list(range(graph.n_nodes))
  
  # best_route becomes better through generations
  for gener in range(n_gener):
    # matrix of newly laid pheromones by current generation
    # pheromones are recomputed at the end of the current generation
    pher_delta = np.zeros(shape=(graph.n_nodes, graph.n_nodes))
    
    # n_ants leaves from each node
    for ant in range(n_ants * graph.n_nodes):
      pos = ant % graph.n_nodes
      
      # controlling current route
      curr_route[0] = pos
      curr_idx = 1
      curr_len = 0
      
      # ant marks visited nodes with truthy values
      visited = np.array([False] * graph.n_nodes)
      
      # iterate through ant steps (their number is n_nodes - 1)
      # the last step is determined by the last and the first nodes in the route
      for step in range(1, graph.n_nodes):
        # mark current node as visited
        visited[pos] = True
        # probablity of moving to each node from the current one
        mv_prob = np.zeros(graph.n_nodes)
        mv_prob_sum = 0
        
        # iterating through all the nodes
        for dest in range(graph.n_nodes):
          # ant won't move to the node where it have already been
          if visited[dest]:
            mv_prob[dest] = 0
            continue
          
          # probability of moving this node
          mv_prob[dest] = (pheromone[pos][dest] ** alfa) * (closeness[pos][dest] ** beta)
          mv_prob_sum += mv_prob[dest]
        
        # to make probabilities sum up to 1
        mv_prob /= mv_prob_sum
        # choose next node randomly with respect to probabilities
        next_pos = np.random.choice(next_pos_choices, p=mv_prob)
        
        # updating current route
        curr_route[curr_idx] = next_pos
        curr_idx += 1
        curr_len += graph.distance(pos, next_pos)
        pos = next_pos
      
      # add the edge from the last to the first node
      curr_len += graph.distance(route[-1], route[0])
      
      # pheromones signals lays by the current ant
      delta = pher_delta_cf / curr_len
      for step in range(1, graph.n_nodes):
        pher_delta[curr_route[step - 1]][curr_route[step]] += delta
      pher_delta[curr_route[-1]][curr_route[0]] += delta
      
      # updating best route
      if curr_len < best_len:
        best_route, curr_route = curr_route, best_route
        best_len = curr_len
        
    # elite ants go only on the best route and lay pheromones there
    elite_delta = floor(n_elites * graph.n_nodes) * pher_delta_cf / best_len
    for step in range(1, graph.n_nodes):
      pher_delta[best_route[step - 1]][best_route[step]] += elite_delta
    pher_delta[best_route[-1]][best_route[0]] += elite_delta
    
    # updating pheromones signals
    for r in range(graph.n_nodes):
      for c in range(graph.n_nodes):
        pheromone[r][c] = min(vapor_cf * pheromone[r][c] + pher_delta[r][c], pher_max)
    
    graph.visualize(best_route, gener)
        
  return best_route

In [78]:
n_popul = 100
route = ants_colony(graph, n_popul=n_popul)
route_evolution_gif(evolution_depth=n_popul, gifname_suffix='_ants')